In [1]:
! mkdir dataset

In [41]:
! pip install theano

  Using cached Theano-1.0.5.tar.gz (2.8 MB)
  Created wheel for theano: filename=Theano-1.0.5-py3-none-any.whl size=2668111 sha256=b58df46598edb05d16ac880b6a9c7e44826af2f9f007cc233eb8fa39db4e7401
  Stored in directory: c:\users\pc\appdata\local\pip\cache\wheels\84\cb\19\235b5b10d89b4621f685112f8762681570a9fa14dc1ce904d9
Successfully built theano


In [3]:
cd dataset

C:\Users\pc\PycharmProjects\DylogT1\ResNet\dataset


In [5]:
pwd

'C:\\Users\\pc\\PycharmProjects\\DylogT1\\ResNet\\dataset'

In [6]:
! kaggle datasets download -d emmarex/plantdisease 


  0%|          | 0.00/658M [00:00<?, ?B/s]
  0%|          | 1.00M/658M [00:00<02:20, 4.92MB/s]
  1%|          | 4.00M/658M [00:00<00:46, 14.6MB/s]
  1%|          | 6.00M/658M [00:00<01:15, 9.11MB/s]
  1%|1         | 8.00M/658M [00:01<01:39, 6.86MB/s]
  2%|1         | 10.0M/658M [00:01<01:58, 5.75MB/s]
  2%|1         | 12.0M/658M [00:01<01:29, 7.55MB/s]
  2%|2         | 14.0M/658M [00:02<01:41, 6.64MB/s]
  2%|2         | 16.0M/658M [00:02<01:22, 8.17MB/s]
  3%|2         | 18.0M/658M [00:02<01:24, 7.92MB/s]
  3%|3         | 20.0M/658M [00:02<01:44, 6.41MB/s]
  3%|3         | 22.0M/658M [00:03<01:27, 7.66MB/s]
  4%|3         | 25.0M/658M [00:03<01:02, 10.7MB/s]
  4%|4         | 27.0M/658M [00:03<01:07, 9.79MB/s]
  4%|4         | 29.0M/658M [00:04<01:34, 6.97MB/s]
  5%|4         | 30.0M/658M [00:04<01:30, 7.25MB/s]
  5%|5         | 33.0M/658M [00:04<01:08, 9.50MB/s]
  5%|5         | 35.0M/658M [00:04<01:28, 7.34MB/s]
  6%|5         | 37.0M/658M [00:04<01:12, 9.00MB/s]
  6%|5         | 39.

In [8]:
import zipfile
with zipfile.ZipFile('plantdisease.zip', 'r') as zip_ref:
    zip_ref.extractall('dataset')

In [51]:
import os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator as IDG

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4681367900673756340
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4163895296
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8950723781917195609
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1660 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [21]:
dat = 'dataset/PlantVillage/'

In [22]:
dir_path = 'dataset/PlantVillage/'

names = []

for dn in os.listdir(dir_path):
    names.append(dn)
print(names)

['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight', 'Potato___healthy', 'Potato___Late_blight', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_healthy', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_mosaic_virus', 'Tomato__Tomato_YellowLeaf__Curl_Virus']


In [43]:
## PARAMETERS
## These varaibles define 

BATCH_SIZE=64
IMG_SIZE=(150, 150)

In [44]:
train_datagen = IDG(
    rescale = 1/.225,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range=20,
    height_shift_range=20,
    rotation_range=0.2,
    preprocessing_function=preprocess_input
    # horizontal_flip=True,
)

valid_datagen = IDG(
    rescale = 1/.225,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range=20,
    height_shift_range=20,
    rotation_range=0.2,
    preprocessing_function=preprocess_input
    # horizontal_flip=True,
)

train_gen = train_datagen.flow_from_directory(
    dat,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
)

valid_gen = valid_datagen.flow_from_directory(
    dat,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
)

Found 20638 images belonging to 15 classes.
Found 20638 images belonging to 15 classes.


In [45]:
class_count = len(list(train_gen.class_indices.keys()))
class_count

15

In [46]:
base_mod = VGG19(weights='imagenet',
                input_shape=(150, 150, 3),
                include_top=False,
                pooling='max',
                classes=class_count,
                classifier_activation='softmax')

base_mod.trainable=False

In [28]:
base_mod.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [47]:
x = base_mod.output
x = Dense(512,activation='relu')(x)
# x = Dense(1024,activation='relu')(x)
# x = Dense(1024,activation='relu')(x)
x = Dense(class_count, activation='softmax')(x)

model= Model(base_mod.input, x)

model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0   

In [52]:
opt = keras.optimizers.Adam(learning_rate=1e-3)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [54]:
history = model.fit(train_gen, epochs=5, validation_data=valid_gen, validation_steps=valid_gen.samples//BATCH_SIZE, steps_per_epoch= train_gen.samples // BATCH_SIZE, verbose=1)

Epoch 1/5
322/322 [==============================] - 183s 569ms/step - loss: 0.6174 - accuracy: 0.8098 - val_loss: 0.5692 - val_accuracy: 0.8232
Epoch 2/5
322/322 [==============================] - 185s 574ms/step - loss: 0.5619 - accuracy: 0.8233 - val_loss: 0.5338 - val_accuracy: 0.8298
Epoch 3/5
322/322 [==============================] - 180s 560ms/step - loss: 0.5183 - accuracy: 0.8348 - val_loss: 0.4965 - val_accuracy: 0.8376
Epoch 4/5
322/322 [==============================] - 177s 549ms/step - loss: 0.4929 - accuracy: 0.8393 - val_loss: 0.4485 - val_accuracy: 0.8497
Epoch 5/5
322/322 [==============================] - 180s 560ms/step - loss: 0.4720 - accuracy: 0.8475 - val_loss: 0.4206 - val_accuracy: 0.8618
